In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from collections import Counter

In [2]:
path = ... # Ваш путь до директории с данными /path/to/data/
data = pd.read_csv(path + 'train_events.csv')
video = pd.read_csv(path + 'video_info.csv')
targets = pd.read_csv(path + 'train_targets.csv')

# Предсказание социально-демографических характеристик пользователей Rutube
    Пользователи RUTUBE не всегда указывают свои данные, такие как возраст и пол, что затрудняет формирование портрета пользователя и создание персонализированных рекомендаций. Это ограничивает возможности платформы в предоставлении контента, который наиболее подходит интересам и потребностям пользователей, тем самым ухудшая пользовательский опыт.

    Необходимо разработать модель, которая на основе истории просмотров сможет предсказывать пол и возраст пользователя. В качестве baseline решения мы предоставляем этот ноутбук.


# Baseline 
    Чтобы предсказать возраст и пол, мы будем использовать медианный возраст и модальное значение пола для просмотренных каналов. Для этого мы подсчитаем статистические данные для каждого author_id на основе обучающей выборки.

    Чтобы оценить качество данного подхода, разделим пользователей с известными социально-демографическими характеристиками на обучающую и валидационную выборки.

In [3]:
data.head(5)

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,512,video_362960,10245341
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,5647,video_96775,10894333
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,71,video_116245,10452976


    В таблице data указаны события пользователей Rutube 
    - event_date : Дата события
    - viewer_uid : Идентификатор пользователя
    - region : Регион пользователя
    - rutube_video_id : Идентификатор видео
    - ua_device_type : Устройство пользователя
    - ua_client_type : Приложение/браузер 
    - ua_os : Операционная система устройства пользователя
    - ua_client_name : Веб-браузер/приложение, с которого пользователь просматривал видео
    - total_watchtime : Время просмотра в секундах

    Выделим валидационную выборку, на которой оценим качество нашего решения. Для этого разделим датасет случайным 
    образом по айди пользователей

In [4]:
TRAIN_IDS, VAL_IDS = train_test_split(data['viewer_uid'].unique(), train_size=0.8, shuffle=True, random_state=11)

In [5]:
train_events = data[data['viewer_uid'].isin(TRAIN_IDS)]
val_events = data[data['viewer_uid'].isin(VAL_IDS)]

In [6]:
val_targets = targets[targets['viewer_uid'].isin(VAL_IDS)]

In [7]:
train_events = train_events.merge(video[['author_id', 'rutube_video_id']], how='left')
train_events = train_events.merge(targets, on='viewer_uid', how='inner')

train_events = train_events.drop(['event_timestamp','rutube_video_id', 'ua_device_type', 'ua_client_type', 'ua_os',\
                                           'ua_client_name', 'total_watchtime'], axis=1)
train_events.head()

,region,viewer_uid,author_id,age,sex
0,Chelyabinsk,10067243,1009219,20,female
1,Bashkortostan Republic,10245341,1006760,40,female
2,St.-Petersburg,10894333,1009257,23,male
3,Moscow,10029092,1058671,41,male
4,Rostov,10013813,1009219,44,female


In [8]:
def get_mode(row): # Кастомная функция вычисления модального значения категориального признака
    counter = Counter(row)
    (most_common, _) = counter.most_common(1)[0]
    
    return most_common

    Группируем данные по идентификаторам авторов и определяем медианный возраст и модальное значение пола среди пользователей, просмотревших видео этого автора.

In [9]:
author_median_age = train_events.groupby('author_id')['age'].median()
author_sex_mode = train_events.groupby('author_id')['sex'].apply(get_mode)

author_sex_mode.name = 'sex_mode'
author_median_age.name = 'median_age'
author_info = author_sex_mode.to_frame().join(author_median_age)

In [10]:
author_info.head()

,sex_mode,median_age
author_id,,
1000003,male,46.0
1000004,male,41.0
1000005,female,34.0
1000007,male,32.0
1000008,male,36.0


In [11]:
val_events = val_events.merge(video[['rutube_video_id', 'author_id']])
val_events = val_events.drop(['event_timestamp','rutube_video_id', 'ua_device_type', 'ua_client_type', 'ua_os',\
                                           'ua_client_name', 'total_watchtime'], axis=1)
val_events.head()

,region,viewer_uid,author_id
0,Moscow,10452976,1020020
1,Kursk Oblast,10084832,1084744
2,Moscow,10043828,1009257
3,St.-Petersburg,10180480,1084744
4,Moscow,10339394,1009257


    Предсказываем целевые переменные тестовых пользователей. 

    На истории просмотренных каналов считаем медианный возраст и моду пола.

In [12]:
val_events = val_events.join(author_info, on='author_id', how='left')
val_events.head()

,region,viewer_uid,author_id,sex_mode,median_age
0,Moscow,10452976,1020020,female,35.0
1,Kursk Oblast,10084832,1084744,female,28.0
2,Moscow,10043828,1009257,female,31.0
3,St.-Petersburg,10180480,1084744,female,28.0
4,Moscow,10339394,1009257,female,31.0


In [13]:
median_age_predict = val_events.groupby('viewer_uid')['median_age'].median()
mode_sex_predict = val_events.groupby('viewer_uid')['sex_mode'].apply(get_mode)

predicts = median_age_predict.to_frame().join(mode_sex_predict)
predicts = predicts.loc[VAL_IDS]


predicts['sex_mode'] = predicts['sex_mode'].fillna(author_info['sex_mode'].mode()[0])
predicts['median_age'] = predicts['median_age'].fillna(author_info['median_age'].median())

age_class_bins = [9, 20, 30, 40, 60] # Возрастные категории пользователей, подробнее в файле с описанием данных
predicts['age_class'] = pd.cut(predicts['median_age'], bins=age_class_bins, labels=[0, 1, 2, 3])
predicts = predicts.reset_index()

In [14]:
predicts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36003 entries, 0 to 36002
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   viewer_uid  36003 non-null  int64   
 1   median_age  36003 non-null  float64 
 2   sex_mode    36003 non-null  object  
 3   age_class   36003 non-null  category
dtypes: category(1), float64(1), int64(1), object(1)
memory usage: 879.3+ KB


In [15]:
predicts.head()

,viewer_uid,median_age,sex_mode,age_class
0,10350073,37.0,female,2
1,10576339,38.0,male,2
2,10112889,31.0,female,2
3,10326819,33.5,female,2
4,10557881,29.0,male,1


In [18]:
val_targets.loc[:, 'age_class'] = pd.cut(val_targets['age'], bins=age_class_bins, labels=[0, 1, 2, 3])

# Подсчет метрики

In [19]:
val_targets = val_targets.sort_values(by='viewer_uid').reset_index(drop=True)
predicts = predicts.sort_values(by='viewer_uid').reset_index(drop=True)

In [20]:
f1_weighted = f1_score(val_targets['age_class'], predicts['age_class'], average='weighted')
accuracy = accuracy_score(val_targets['sex'], predicts['sex_mode'])

final_score = 0.7 * f1_weighted + 0.3 * accuracy
print(f'Weighted F1 = {f1_weighted:.4f} \nAccuracy = {accuracy:.4f} \nFinal Score = {final_score:.4f}')

Weighted F1 = 0.3630 
Accuracy = 0.7330 
Final Score = 0.4740


# Пример заполнения submission.csv
    Перед отправкой убедитесь, что количество уникальных пользователей в тестовой выборке и в вашей таблице предсказаний совпадает. Проверьте, чтобы в столбцах с прогнозируемыми значениями не было пропущенных значений. 

    Идентификаторы пользователей (viewer_uid) нужно сохранить как отдельную колонку, а не индекс таблицы.
    
    Не забудьте сохранить файл с параметром index = False.

In [21]:
submission = pd.DataFrame(columns=['viewer_uid', 'sex', 'age_class'])
submission['viewer_uid'] = predicts['viewer_uid'].values
submission['sex'] = predicts['sex_mode'].values
submission['age_class'] = predicts['age_class'].values

In [22]:
submission.head()

,viewer_uid,sex,age_class
0,10000001,female,2
1,10000007,male,2
2,10000010,male,2
3,10000012,male,2
4,10000014,male,2


In [ ]:
submission.to_csv(
    # path/submission.csv,
    # index=False 
)